<a href="https://colab.research.google.com/github/PavankumarUppar/DLL/blob/main/Program5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim

# Create a synthetic dataset for demonstration purposes
# Generate random stock prices
num_samples = 1000
seq_length = 10  # Sequence length for each input
num_features = 1  # Only using the stock price as a feature

# Creating synthetic stock price data
stock_prices = np.random.randn(num_samples)

# Prepare the data
input_data = []
target_data = []

for i in range(len(stock_prices) - seq_length):
    input_data.append(stock_prices[i:i + seq_length])
    target_data.append(stock_prices[i + seq_length])

input_data = np.array(input_data)
target_data = np.array(target_data)

# Convert data to PyTorch tensors
input_tensor = torch.from_numpy(input_data).unsqueeze(-1).float()
target_tensor = torch.from_numpy(target_data).unsqueeze(-1).float()

# Define a GRU-based model
class StockPriceGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(StockPriceGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])  # Taking only the last output
        return out

# Define hyperparameters
input_size = num_features
hidden_size = 64
num_layers = 1
learning_rate = 0.001
num_epochs = 100

# Instantiate the model, define loss function and optimizer
model = StockPriceGRU(input_size, hidden_size, num_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    outputs = model(input_tensor)
    loss = criterion(outputs, target_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# Predictions
model.eval()
with torch.no_grad():
    test_input = input_tensor[-1].unsqueeze(0)  # Taking the last sequence for prediction
    predicted = model(test_input)
    print(f"Predicted price: {predicted.item()}")


Epoch [10/100], Loss: 0.945843
Epoch [20/100], Loss: 0.945254
Epoch [30/100], Loss: 0.945132
Epoch [40/100], Loss: 0.944545
Epoch [50/100], Loss: 0.944425
Epoch [60/100], Loss: 0.944218
Epoch [70/100], Loss: 0.943930
Epoch [80/100], Loss: 0.943528
Epoch [90/100], Loss: 0.942899
Epoch [100/100], Loss: 0.941879
Predicted price: 0.039187751710414886
